In [1]:
from Envs.pendulum import PendulumEnv

In [2]:
env = PendulumEnv()

In [3]:
# import gymnasium as gym
# env = gym.make("CartPole-v0")

In [4]:
env.reset()

(array([-0.8742544 , -0.4854681 ,  0.85613537], dtype=float32), {})

In [5]:
from stable_baselines3 import PPO

In [6]:
PPO_Policy = PPO("MlpPolicy", env)

In [7]:
PPO_Policy.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
obs, info = env.reset()

In [9]:
obs

array([ 0.67721   , -0.73578984,  0.50694984], dtype=float32)

In [10]:
PPO_Policy.predict(obs)[0]

array([-0.15732887], dtype=float32)

In [11]:
PPO_Policy.policy.

SyntaxError: invalid syntax (4254809501.py, line 1)

In [12]:
from stable_baselines3 import PPO
import torch

In [13]:
from MBEnvs.mb_pendulum import MB_PendulumEnv
env_models = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MB_env = MB_PendulumEnv(env_models, device)

In [14]:
Global_RL = PPO("MlpPolicy", MB_env)

In [15]:
from Envs.pendulum import PendulumEnv

In [16]:
from Client import *
from Server import PPO_Server


In [17]:
CLIENTS_NUM = 3

env_models = []
real_envs = []
Clients = []
for i in range(CLIENTS_NUM):
    real_envs.append( PendulumEnv())
    policy_net = Global_RL
    agent = BaseAgent(policy_net)
    client = FRLClient(real_envs[i], agent, lr = 3e-4, hidden_size = 256, device = "cuda")
    env_model = copy.deepcopy(client.model)
    Clients.append(client)
    env_models.append(env_model)
        

In [18]:
MB_env.models = env_models

In [19]:
obs, info = MB_env.reset()
obs

array([ 0.70778865,  0.7064243 , -0.14476484], dtype=float32)

In [20]:
action = MB_env.action_space.sample()
action

array([1.1462978], dtype=float32)

In [21]:
next_obs, reward, terminal, trun, info = MB_env.step(MB_env.action_space.sample())
next_obs, reward, terminal, trun, info

(array([-0.04459275,  0.08195187, -0.01005399], dtype=float32),
 -0.6194905042648315,
 False,
 False,
 {})

In [22]:
MB_env._recover_theta(obs[0], obs[1])

0.7844334

In [23]:
MB_env.state

array([ 0.78443341, -0.14476483])

In [24]:
MB_env.cur_model

PredictionModel(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=3, bias=True)
  )
)

In [25]:
client.sample_data(n=1000)

TypeError: '>=' not supported between instances of 'NoneType' and 'float'

In [14]:
from stable_baselines3.common.env_checker import check_env

In [15]:
check_env(MB_env)

D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\env_checker.py:428: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [11]:
MB_env.cur_model.net

Sequential(
  (0): Linear(in_features=4, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=3, bias=True)
)

In [12]:
obs_tensor = torch.tensor(obs, dtype=torch.float32)
action_tensor = torch.tensor(action, dtype=torch.float32)

In [13]:
input_d = torch.cat((obs_tensor, action_tensor))
input_d

tensor([-0.5031, -0.8642,  0.0275,  0.6901])

In [14]:
output_tensor = MB_env.cur_model.net.forward(input_d)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [20]:
input_d.to('cuda')

tensor([-0.9941, -0.1089, -0.3072, -1.3609], device='cuda:0')

In [24]:
MB_env.cur_model.cuda()
output_tensor = MB_env.cur_model.net.forward(input_d)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [38]:
MB_env.cur_model.net.cpu()

Sequential(
  (0): Linear(in_features=4, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=3, bias=True)
)

In [55]:
output_tensor.detach().numpy()

array([ 0.06205782, -0.00997211, -0.05215619], dtype=float32)

In [15]:
for param in MB_env.cur_model.net.parameters():
    print(param.device)

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [16]:
test_model = PredictionModel(3,1,64)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
test_model.to(device)

PredictionModel(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [24]:
input_test = torch.cat((obs_tensor, action_tensor))

In [30]:
input_test = input_test.to(device)

In [31]:
input_test

tensor([-0.5031, -0.8642,  0.0275,  0.6901], device='cuda:0')

In [32]:
test_model.forward(input_test)

tensor([-0.1828,  0.0928,  0.1543], device='cuda:0', grad_fn=<ViewBackward0>)

In [27]:
input_test.to(device)

tensor([-0.5031, -0.8642,  0.0275,  0.6901], device='cuda:0')

In [28]:
test_model.forward(input_test)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [1]:
from Client import *

In [ ]:
FRLCliente()

In [1]:
from MBEnvs.mb_pendulum_emb import MB_PendulumEnv

In [4]:
from PredictionModel import PredictionModel
from H_Envs.pendulum import PendulumEnv
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import numpy as np
import torch
# from sb3_contrib import TRPO
from Agent import SB3Agent
import copy

from H_Envs.pendulum_emb import PendulumEnvEmb

In [5]:
env_paras = [7.0, 10.0, 13.0]

In [6]:
# 将每个元素转换为 NumPy 数组
env_paras_arrays = [np.array([x]) for x in env_paras]

In [7]:
env_paras_arrays 

[array([7.]), array([10.]), array([13.])]

In [8]:
real_envs = []
eva_envs = []
Clients = []
for i in range(len(env_paras)):
    real_envs.append( TimeLimit(PendulumEnv(g=env_paras[i]), max_episode_steps=200) )
    eva_envs.append( TimeLimit(PendulumEnvEmb(g=env_paras[i]), max_episode_steps=200) )

In [7]:
real_envs[0].g

D:\Anaconda\envs\py3.8\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.g to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.g` for environment variables or `env.get_wrapper_attr('g')` that will search the reminding wrappers.
  logger.warn(


7.0

In [8]:
from Client_diff_Emb import FRLClient

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
env_paras = [7.0, 10.0, 13.0]
embs = [np.array([x]) for x in env_paras]

CLIENTS_NUM = len(env_paras)
env_models = []
MB_env = TimeLimit(MB_PendulumEnv(env_models,embs,device), max_episode_steps = 200)
    
    # Global_RL = PPO("MlpPolicy", MB_env, verbose=1)
Global_RL = TRPO("MlpPolicy", MB_env, verbose=1)
    
    # env_theta = [0.1, 0.3, 0.5, 0.7, 0.9]
real_envs = []
eva_envs = []
Clients = []
for i in range(CLIENTS_NUM):
    real_envs.append( TimeLimit(PendulumEnv(g=env_paras[i]), max_episode_steps=200) )
    eva_envs.append( TimeLimit(PendulumEnvEmb(g=env_paras[i]), max_episode_steps=200) )
    policy_net = Global_RL
    agent = SB3Agent(policy_net)
    client = FRLClient(real_envs[i], agent, lr = 3e-4, hidden_size = 256, device = device, emb=embs[i])
    Clients.append(client)
    env_model = copy.deepcopy(client.model)
    env_models.append(env_model)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
Clients[0].emb

array([7.])

In [12]:
MB_env.env_embs

D:\Anaconda\envs\py3.8\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.env_embs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.env_embs` for environment variables or `env.get_wrapper_attr('env_embs')` that will search the reminding wrappers.
  logger.warn(


[array([7.]), array([10.]), array([13.])]

In [13]:
env_models

[PredictionModel(
   (net): Sequential(
     (0): Linear(in_features=4, out_features=256, bias=True)
     (1): ReLU()
     (2): Linear(in_features=256, out_features=256, bias=True)
     (3): ReLU()
     (4): Linear(in_features=256, out_features=3, bias=True)
   )
 ),
 PredictionModel(
   (net): Sequential(
     (0): Linear(in_features=4, out_features=256, bias=True)
     (1): ReLU()
     (2): Linear(in_features=256, out_features=256, bias=True)
     (3): ReLU()
     (4): Linear(in_features=256, out_features=3, bias=True)
   )
 ),
 PredictionModel(
   (net): Sequential(
     (0): Linear(in_features=4, out_features=256, bias=True)
     (1): ReLU()
     (2): Linear(in_features=256, out_features=256, bias=True)
     (3): ReLU()
     (4): Linear(in_features=256, out_features=3, bias=True)
   )
 )]

In [14]:
MB_env = TimeLimit(MB_PendulumEnv(env_models,embs,device), max_episode_steps = 200)

In [10]:
from stable_baselines3.common.env_checker import check_env

In [11]:
check_env(MB_env)

NameError: name 'MB_env' is not defined

In [12]:
eva_envs[0]

<TimeLimit<PendulumEnvEmb instance>>

In [13]:
check_env(eva_envs[0])

D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\env_checker.py:428: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [19]:
np.array([7.0])

array([7.])